<a href="https://colab.research.google.com/github/baibai25/survey/blob/main/faiss_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -V

Python 3.7.11


In [2]:
!wget https://anaconda.org/pytorch/faiss-gpu/1.7.1/download/linux-64/faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2
!tar xvjf faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2

--2021-08-19 11:53:01--  https://anaconda.org/pytorch/faiss-gpu/1.7.1/download/linux-64/faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9cc393ec612061f033a/60c02fb0dcb792e0caecd9e8?response-content-disposition=attachment%3B%20filename%3D%22faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20210819T115302Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEHcaCXVzLWVhc3QtMSJIMEYCIQD5OncGAW38Y2y2QFu84yNd7OdKUcpQZjCpNc%2F6y3SpRQIhAOwyw5rk2SUvwFWYKXjs3gcrM%2FN6XRZSWCk%2BFeZ%2BAwZCKoMECKD%2F

In [3]:
!cp -r lib/python3.7/site-packages/* /usr/local/lib/python3.7/dist-packages/
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 6.9 kB/s 


In [4]:
import faiss

In [5]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

In [6]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [7]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [8]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [9]:
#  This isn't a particularly large number, so let's pull in a few more similar datasets.
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [10]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [11]:
len(set(sentences))

14505

In [12]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [13]:
!pip install sentence-transformers

     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 2.6 MB 60.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 895 kB 74.3 MB/s 
     |████████████████████████████████| 636 kB 77.7 MB/s 
     |████████████████████████████████| 3.3 MB 53.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=a7827def559f09878f9069614a4bd924abc6eac4710441338ca04ab025774e48
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

(14504, 768)

In [15]:
d = sentence_embeddings.shape[1]
d

768

In [16]:
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [17]:
index.add(sentence_embeddings)
index.ntotal

14504

In [18]:
# Then search given a query xq and number of nearest neigbors to return k.
k = 4
xq = model.encode(["Someone sprints with a football"])

In [19]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[1289  258 6465 9814]]
CPU times: user 35.4 ms, sys: 142 µs, total: 35.5 ms
Wall time: 21.1 ms


In [20]:
for i in I.flatten():
  print(sentences[i])

A group of football players is running in the field
A group of people playing football is running in the field
Two groups of people are playing football
A person playing football is running past an official carrying a football


In [21]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [22]:
vecs.shape

(4, 768)

In [23]:
vecs[0][:100]

array([ 0.01627044,  0.22325914, -0.15037398, -0.30747244, -0.27122441,
       -0.10593162, -0.06460952,  0.0473813 , -0.73349017, -0.37657687,
       -0.76762819,  0.1690287 ,  0.53107637,  0.51176649,  1.14415872,
       -0.08562911, -0.67240077, -0.96637076,  0.02545413, -0.2155984 ,
       -1.25656617, -0.82982153, -0.09824991, -0.21850836,  0.50610226,
        0.10527953,  0.50396866,  0.65242958, -1.39458752,  0.65847486,
       -0.21525307, -0.22487493,  0.81818324,  0.08464275, -0.76141673,
       -0.28928292, -0.09825823, -0.7304619 ,  0.07855775, -0.84354633,
       -0.59242088,  0.7747137 , -1.20920539, -0.22757916, -1.30733597,
       -0.23081459, -1.31322551,  0.01629054, -0.97285467,  0.19308199,
        0.4742457 ,  1.18920863, -1.96741307, -0.70061159, -0.29638687,
        0.6053372 ,  0.6240744 , -0.70340389, -0.86754251,  0.17673193,
       -0.19170511, -0.02951988,  0.22623582, -0.16695465, -0.80402541,
       -0.4591893 ,  0.69675535, -0.2492822 , -1.0147866 , -0.92

In [24]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [25]:
index.is_trained

False

In [26]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [27]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [28]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[1289  258 6465 9814]]
CPU times: user 2.01 ms, sys: 117 µs, total: 2.13 ms
Wall time: 2.62 ms


In [29]:
# We can increase the number of nearby cells to search too with nprobe.
index.nprobe = 10

In [30]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[1289  258 6465 9814]]
CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 11.4 ms


In [31]:
index.make_direct_map()
index.reconstruct(7460)[:100]

array([ 0.50945   , -1.4041907 ,  0.20849149,  0.0349403 ,  0.17578006,
       -0.02460578, -0.43026814, -0.08036996, -0.39414254, -0.30097944,
        0.16127892,  0.08792005,  0.6994182 ,  0.24138494,  1.5857044 ,
       -0.29900795, -0.27878004, -0.36117572,  0.36481982, -0.48925608,
       -1.3655338 ,  0.6661032 , -0.18052626, -0.384278  ,  0.20996924,
       -0.7218925 ,  0.5373777 ,  0.07798336, -0.38038802,  0.54617465,
       -0.57938737, -0.15493822,  0.09492378,  0.28323665,  0.18942732,
       -0.1384364 , -0.6062889 , -0.9664947 , -0.2511334 ,  0.20462286,
       -1.0837119 , -0.33464327,  0.22415146, -0.37536067, -0.88055545,
       -0.48878542,  0.9981171 ,  0.5281094 , -0.10889211, -0.01945428,
        0.64730334, -0.8937543 , -0.11548422,  1.5307478 ,  0.3181488 ,
        0.60308397, -0.07156699, -0.64043367, -0.20694101,  0.43047565,
        1.0068098 , -0.46007603, -0.30239436,  0.22375421, -0.7635119 ,
        1.13122   ,  0.846698  , -0.13843797, -0.6577174 , -0.82

In [32]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [33]:
index.is_trained

False

In [34]:
index.train(sentence_embeddings)

In [35]:
index.add(sentence_embeddings)

In [36]:
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value

In [37]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 258 1581 2456 1289]]
CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 8.56 ms


In [38]:
[f'{i}: {sentences[i]}' for i in I[0]]

['258: A group of people playing football is running in the field',
 '1581: Different teams are playing football on a field',
 '2456: A man in a football uniform is running with a football during a game.',
 '1289: A group of football players is running in the field']